In [1]:
%pip install openai
import sys
print("Python executable:", sys.executable)
print("OpenAI installation check:")
try:
    import openai
    print("✓ OpenAI successfully imported")
    print("OpenAI version:", openai.__version__)
except ImportError as e:
    print("✗ OpenAI import failed:", e)

Note: you may need to restart the kernel to use updated packages.
Python executable: /Users/jennyhe/miniconda3/envs/llm/bin/python
OpenAI installation check:
✓ OpenAI successfully imported
OpenAI version: 2.6.1


In [ ]:
# openai.api_key = "sk-proj-1234567890"


In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

### Two Principles for Prompting:

1. **Write clear and specific instructions**
     - Tactic 1: Use delimiters, such as:
       - Triple quotes: """
       - Triple backticks: ```
       - Triple dashes: ---
       - Angle brackets: < >
       - XML tags: <tag> </tag>
     - Tactic 2: Ask for structured output
       - HTML, JSON
     - Tactic 3: Check whether conditions are satisfied
       - check assumptions required to do the task
     - Tactic 4: Few-shot prompting
       - Give successful examples of completing tasks, then ask model to perform the task


In [ ]:
# example for tactic 1
text = f"""
You should express what you want a model to do by providing instructions that are clear and specific as you can possibly make them. This will guide the model towards the desired output, and reduce the chances of receiving an irrelevant or incorrect response. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)




In [ ]:
# example for tactic 2
prompt = f""" 
Generate a list of three made-up book titles along with their authors and genre.
Provide them in a JSON format with the following keys: book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)




In [ ]:
# example for tactic 3
text = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, grab a cup and put a tea bag in it. Once the water is hot enough, turn off the heat and pour the water over the tea bag. Let it sit for a bit so the tea can brew. After a few minutes, take out the tea bag. If you like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious cup of tea to enjoy.
"""
prompt = f"""
You will be provided with a text delimited by triple quotes.
If it contains a sequence of instructions, rewrite those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, just output "No instructions found".

\"\"\"
{text}
\"\"\"
"""
response = get_completion(prompt)
print("Compeletion for text:" )
print(response)

In [ ]:
# example for tactic 3 - continuation
text = f"""
The sun shines brightly in a cloudless blue sky, while a gentle breeze carries the light scent of blooming flowers. The warmth of the day is perfectly balanced by the soft wind, making it a delightful and peaceful atmosphere. Everything feels calm and full of promise under this beautiful, fair-weather day.
"""
prompt = f"""
You will be provided with a text delimited by triple quotes.
If it contains a sequence of instructions, rewrite those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, just output "No instructions found".

\"\"\"
{text}
\"\"\"
"""
response = get_completion(prompt)
print("Compeletion for text:" )
print(response)


In [ ]:
# example for tactic 4
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

2. **Give the model time to think**
    - Tactic 1: Specify the steps to complete a task
       - Step 1:...
       - Step 2:...
       - ...
       - Step N:...
    - Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
text = f"""
In a charming village, sibling Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck - Jack tripped on a stone and tumbled down the hill, with Jill following suit.
Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirit remained undimmed, and they continued exploring with delight.
"""
prompt_1 = f"""
Performing the following actions:
1 - Summarize the following text delimited by triple brackticks with 1 sentence.
2 - Translate the summary into Spanish.
3 - List each name in the Spanish summary.
4 - Output a json object with the following keys: Spanish_summary, num_names.

Separate your answer with line breaks.
Text:
```{text}```
"""
response_1 = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response_1)





In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
 me a flat $100k per year, and an additional $10 / square \
 foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not.
- Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
---
question here
---
Student's solution:
---
student's solution here
---
Actual solution:
---
steps to work out the solution and your solution here
---
Is the student's solution the same as actual solution just calculated:
---
yes or no
---
Student grade:
---
correct or incorrect
---

Question:
---
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
---

Student's solution:
---
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
---
Actual solution:
"""
response = get_completion(prompt)
print(response)






### Model Limitations:

1. ** Hallucination
   - Makes statements that sound plausible but are not true
  
2. ** Reducing hallucinations:
   - First find relevant information, then answer the question based on the relevant information.